In [2]:
import os
import numpy as np
from PIL import Image

# -----------------------------
# 1. PARAMETERS
# -----------------------------

image_size = (64,64) # Resizes all the loaded images into a size of 64X64 pixels 
data_directory = "Images" # Specifies the root directory containing the image sub-folders (ie. "Dogs" and "Cats")

# -----------------------------
# 2. LOAD IMAGES
# -----------------------------
def loading_images(data_directory,mode, image_size):

    """
    Loads images from the specified directory, resizes them, flattens them into vectors,
    and assigns labels based on folder names.

    Assumes each class has a subfolder with 'train' images.

    Arguments:
    data_directory -- string, path to the root directory containing class folders
    mode -- string, either 'train' or 'test' to specify which subfolder to load
    image_size -- tuple(int, int), size to which each image will be resized (width, height)

    Returns:
    X -- list of numpy arrays, each array is a flattened image vector
    y -- list of integers, labels corresponding to each image
    label_dict-- dictionary mapping class names (folder names) to integer labels, e.g., {'Dogs': 0, 'Cats': 1}
    """
    X = [] # Empty list for storing flattened image vectors 
    y = [] # Empty list for storing interger labels (0 and 1)
    
    # Get class labels from folder names and map to integers
    
    labels = os.listdir(data_directory) # lists all the sub-folders of the specified root directory, that contain the images
    label_dict = {label: i for i, label in enumerate(labels)}
    
    # Loop through each class folder and creates path for the mode
    for label in labels:
        folder = os.path.join(data_directory, label, mode)
        if not os.path.isdir(folder):
            continue  # Skip if train/test folder doesn't exist

        Files = os.listdir(folder)
        for filename in Files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder, filename)
               # Open image and convert to RGB
                img = Image.open(image_path).convert("RGB")
              # Resizes the loaded image into the size specified above (64x64 pixels)
                img_resized = img.resize(image_size)
              # Creates a numpy array from the resized image
                X_array = np.array(img_resized)
             # Flattens the array into a vector and appends it into X
                X.append(X_array.flatten())
             # Appends each class label into a list "y"
                y.append(label_dict[label])

    # Converts X into an array and normalizes all pixel values to [0,1]
    X_norm = (np.array(X))/255.0
    # Converts y labels into an array
    y_array = np.array(y)
    
    return X_norm, y_array, label_dict
    

In [3]:

# Assigning the outputs of the "loading_images" function to their respective variables
X_train, y_train, label_dict = loading_images("../Images", "train", image_size)
X_test, y_test = loading_images("../Images", "test", image_size)[:2]

# Transpose for math: changing dimensions to (features, training_examples)
X_train, X_test = X_train.T, X_test.T
y_train, y_test = y_train.reshape(1, y_train.shape[0]), y_test.reshape(1, y_test.shape[0])

In [222]:
# ------------------------------------
# 3. PARAMETER INITIALIZATION FUNCTION
# ------------------------------------

def parameter_init(input_size, hidden_1, hidden_2, hidden_3, output_size):
    """Initializes weights and biases for a 4-layer neural network.
    
    Arguments:    
    input_size -- int, number of input features (size of the input layer)
    hidden_1 -- int, number of units in the first hidden layer
    hidden_2 -- int, number of units in the second hidden layer
    hidden_3 -- int, number of units in the third hidden layer
    output_size -- int, number of units in the output layer
    
    Returns:
    parameters -- dict containing initialized weights (W1, W2, W3, W4) 
                  and biases (b1, b2, b3, b4) for each layer
    """
    np.random.seed(1)

    # Weights: initialized with small random values (scaled by 0.01)
    # to prevent exploding activations and help stable learning.
    # Biases: initialized to zero.
    W1 = np.random.randn(hidden_1, input_size)*np.sqrt(2. / input_size)
    b1 = np.zeros((hidden_1, 1))
    W2 = np.random.randn(hidden_2, hidden_1)*np.sqrt(2. / hidden_1)
    b2 = np.zeros((hidden_2,1))
    W3 = np.random.randn(hidden_3, hidden_2)*np.sqrt(2. / hidden_2)
    b3 = np.zeros((hidden_3,1))
    W4 = np.random.randn(output_size, hidden_3)*np.sqrt(1 / hidden_3)
    b4 = np.zeros((output_size,1))
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3":W3, "b3":b3, "W4":W4, "b4":b4}
    return parameters
    

# -----------------------------
# 4. ACTIVATION FUNCTIONS
# -----------------------------

def relu(Z):
    A = np.maximum(0,Z)
    return A
def relu_derivative(Z):
    return (Z>0).astype(float)
def sigmoid(Z4):
    Z4 = np.clip(Z4, -500, 500)
    y_hat = 1/(1 + np.exp(-Z4))
    return y_hat

def sigmoid_derivative(y_hat):
    y_hat_derivative = y_hat*(1-y_hat)
    return y_hat_derivative

# -------------------------------
# 5. FORWARD BACKWARD PROPAGATION
# -------------------------------

# Before calling this function, make sure to initialize parameters:
# parameters = parameter_init(input_size, hidden_1, hidden_2, hidden_3, output_size)
def forward_prop(X_train, parameters):

    # Retrieve weights and biases from dictionary
    W1, b1 = parameters["W1"], parameters["b1"]
    W2, b2 = parameters["W2"], parameters["b2"]
    W3, b3 = parameters["W3"], parameters["b3"]
    W4, b4 = parameters["W4"], parameters["b4"]

    Z1 = W1.dot(X_train) + b1
    A1 = relu(Z1)  
    Z2 = W2.dot(A1) + b2
    A2 = relu(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = relu(Z3)
    Z4 = W4.dot(A3) + b4
    y_hat = sigmoid (Z4)

    cache = {"Z1":Z1, "A1": A1, "Z2": Z2, "A2": A2, "Z3": Z3,"A3": A3,"Z4": Z4, "A4": y_hat}
    return cache

def Cost_function(y, y_hat):
    m = y.shape[1] # number of training examples

    eps = 1e-15
    y_hat = np.clip(y_hat, eps, 1 - eps)
    cost = -np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)) / m
    cost = np.squeeze(cost) 
    return cost
    
# -----------------------
# 5. BACKWARD PROPAGATION
# -----------------------

def backward_prop(parameters, cache, X_train, y_train):
    W1, W2, W3, W4 = parameters["W1"], parameters["W2"], parameters["W3"], parameters["W4"]
    A1, A2, A3, A4 = cache["A1"], cache["A2"], cache["A3"], cache["A4"]
    Z1, Z2, Z3, Z4 = cache["Z1"], cache["Z2"], cache["Z3"], cache["Z4"]
    
    m = X_train.shape[1]

    # Layer 4 (output layer)
    dZ4 = A4 - y_train
    dW4 = (1/m) * np.dot(dZ4, A3.T)
    db4 = (1/m) * np.sum(dZ4, axis=1, keepdims=True)

    # Hidden Layer 3
    dZ3 = np.dot(W4.T, dZ4) * relu_derivative(Z3)
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

    # Hidden Layer 2
    dZ2 = np.dot(W3.T, dZ3) * relu_derivative(Z2)
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    # Hidden Layer 1
    dZ1 = np.dot(W2.T, dZ2) * relu_derivative(Z1)
    dW1 = (1/m) * np.dot(dZ1, X_train.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    derivatives = {
        "dW4": dW4, "db4": db4,
        "dW3": dW3, "db3": db3,
        "dW2": dW2, "db2": db2,
        "dW1": dW1, "db1": db1
    }
    
    return derivatives
# -----------------------------
# 6. PARAMETER UPDATE
# -----------------------------

def parameter_update(parameters, derivatives, learning_rate = 1.2):

    L = len(parameters) // 2  # number of layers
    for l in range(1, L + 1):
        print(f"Layer {l}: mean|dW| = {np.mean(np.abs(derivatives[f'dW{l}'])):.6f}, "
          f"mean|db| = {np.mean(np.abs(derivatives[f'db{l}'])):.6f}")
        parameters[f"W{l}"] -= learning_rate * derivatives[f"dW{l}"]
        parameters[f"b{l}"] -= learning_rate * derivatives[f"db{l}"]
    return parameters

In [223]:
# -------------------------------
# 7. TRAINING
# -------------------------------

def train(X, y, hidden_sizes = [128, 64, 32], lr=0.0005, epochs=500):
    parameters = parameter_init(12288,hidden_sizes[0],hidden_sizes[1],hidden_sizes[2],1)
    
    for i in range(epochs):
        
    # 3.1 Forward propagation
        cache = forward_prop(X_train, parameters)
        y_hat = cache['A4']

        cost = Cost_function(y, y_hat)
        
        derivatives = backward_prop(parameters, cache, X, y)

        parameters = parameter_update(parameters, derivatives)

        if i % 100 == 0:
            print(f"Epoch {i}, Loss: {cost:.4f}")


    return parameters

In [224]:
train(X_train, y_train)

Layer 1: mean|dW| = 0.002253, mean|db| = 0.003820
Layer 2: mean|dW| = 0.001916, mean|db| = 0.004492
Layer 3: mean|dW| = 0.002218, mean|db| = 0.006407
Layer 4: mean|dW| = 0.029307, mean|db| = 0.086175
Epoch 0, Loss: 0.6980
Layer 1: mean|dW| = 0.008534, mean|db| = 0.015203
Layer 2: mean|dW| = 0.145517, mean|db| = 0.017080
Layer 3: mean|dW| = 0.299963, mean|db| = 0.032954
Layer 4: mean|dW| = 8.352918, mean|db| = 0.500000
Layer 1: mean|dW| = 7.257044, mean|db| = 12.431956
Layer 2: mean|dW| = 70.162012, mean|db| = 3.178553
Layer 3: mean|dW| = 161.405304, mean|db| = 0.845081
Layer 4: mean|dW| = 809.755454, mean|db| = 0.500000
Layer 1: mean|dW| = 15130.662358, mean|db| = 26983.354832
Layer 2: mean|dW| = 94514.756234, mean|db| = 299.723522
Layer 3: mean|dW| = 7194293.216466, mean|db| = 25.506937
Layer 4: mean|dW| = 2317079.228113, mean|db| = 0.500000
Layer 1: mean|dW| = 0.000000, mean|db| = 0.000000
Layer 2: mean|dW| = 0.000000, mean|db| = 2121213775.049297
Layer 3: mean|dW| = 318645.635834, m

{'W1': array([[-0.00017422, -0.02849526, -0.02652017, ..., -0.03093749,
         -0.02735309, -0.01156564],
        [-0.01477295, -0.02468144, -0.03533209, ..., -0.01956735,
         -0.03297317, -0.02412673],
        [-0.00988243, -0.00766312,  0.00328957, ...,  0.00798128,
         -0.00297322, -0.01062378],
        ...,
        [-0.02033536,  0.00335188,  0.00034445, ..., -0.00934644,
         -0.01679735, -0.01637165],
        [ 0.02234594,  0.00417467,  0.01646069, ..., -0.00569442,
         -0.00170059, -0.00356538],
        [-0.02910705, -0.02724262, -0.03291246, ..., -0.05738522,
          0.00114862, -0.01176337]]),
 'b1': array([[-4.23299852e-02],
        [-4.97456628e-02],
        [-6.85942448e-03],
        [-3.78382297e-04],
        [-5.27205763e-03],
        [ 0.00000000e+00],
        [-2.52393351e-03],
        [-9.04863786e-02],
        [-1.67430981e-04],
        [-2.62294627e-03],
        [-9.70963599e-02],
        [-3.45373369e-03],
        [-2.82121963e-02],
        [-

In [105]:
parameters=parameter_init(12288,128,64,32,1)

In [109]:
W2_before = parameters["W2"].copy()
parameters = parameter_update(parameters, derivatives, learning_rate=1.2)
W2_after = parameters["W2"]

print("Max change in W1:", np.max(np.abs(W1_after - W1_before)))

Layer 1: mean|dW| = 0.003552, mean|db| = 0.006000
Layer 2: mean|dW| = 0.003115, mean|db| = 0.007532
Layer 3: mean|dW| = 0.003795, mean|db| = 0.011754
Layer 4: mean|dW| = 0.038013, mean|db| = 0.119673
Max change in W1: 0.07966604463511338


In [93]:
cache = forward_prop(X_train, parameters)

In [102]:
derivatives = backward_prob(parameters, cache, X_train, y_train )
parameter_update(parameters, derivatives)

Layer 1: mean|dW| = 0.003552, mean|db| = 0.006000
Layer 2: mean|dW| = 0.003115, mean|db| = 0.007532
Layer 3: mean|dW| = 0.003795, mean|db| = 0.011754
Layer 4: mean|dW| = 0.038013, mean|db| = 0.119673


{'W1': array([[ 0.02889013,  0.00053516,  0.00181529, ...,  0.00063127,
          0.00248862,  0.01869865],
        [ 0.01479205,  0.00482789, -0.00656446, ...,  0.01124901,
         -0.00381177,  0.00496795],
        [-0.00776768, -0.00560723,  0.00523111, ...,  0.00898555,
         -0.00232056, -0.00973639],
        ...,
        [-0.01919107,  0.0044938 ,  0.00145654, ..., -0.00715212,
         -0.01475998, -0.01429131],
        [ 0.02341782,  0.00506834,  0.01722273, ..., -0.00477758,
         -0.00085054, -0.00276448],
        [ 0.00686845,  0.00876229,  0.00233785, ..., -0.01925701,
          0.03737518,  0.02437289]]),
 'b1': array([[ 1.15978291e-02],
        [ 3.26759120e-03],
        [-4.12537040e-03],
        [-2.32972815e-04],
        [-2.27449749e-03],
        [ 0.00000000e+00],
        [-2.43642136e-03],
        [ 3.72203567e-03],
        [-1.00932519e-04],
        [-1.41577517e-03],
        [ 6.28614079e-03],
        [-2.14263787e-03],
        [ 5.27358172e-03],
        [-

In [ ]:
backward_prob()

In [126]:
parameters=parameter_init(12288,128,64,32,1)
y_hat = forward_prop(X_train, parameters)[0]

KeyError: 0

In [127]:
parameters

{'W1': array([[ 0.02072301, -0.00780464, -0.00673829, ..., -0.00672607,
         -0.00485228,  0.01211237],
        [ 0.01287553,  0.00293287, -0.00847242, ...,  0.00880105,
         -0.00622989,  0.00264863],
        [-0.0045461 , -0.0024939 ,  0.00812418, ...,  0.01049646,
         -0.00133223, -0.00839405],
        ...,
        [-0.01829951,  0.00538994,  0.00236238, ..., -0.00469666,
         -0.01246908, -0.01194158],
        [ 0.02354356,  0.00503393,  0.01707549, ..., -0.00479893,
         -0.00092674, -0.00287958],
        [ 0.00663182,  0.00857529,  0.00216533, ..., -0.0196245 ,
          0.03708781,  0.0241416 ]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.

In [6]:
Cost_function(y_train, y_hat)

0.6931441807084028

In [7]:
y_hat

array([[0.50002217, 0.50000296, 0.5000125 , 0.50001995, 0.50002182,
        0.50001195, 0.50000897, 0.50000907, 0.50001882, 0.50003302,
        0.50003582, 0.50001238, 0.50002145, 0.500018  , 0.50000161,
        0.50000136, 0.50002337, 0.50002958, 0.5000227 , 0.50002496]])

In [8]:
y_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])